# TruBot testing

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [2]:
from trulens_eval.keys import *
from trulens_eval.slackbot import get_or_make_chain, get_answer
from trulens_eval.util import TP
from trulens_eval import Tru
from trulens_eval.tru_feedback import Huggingface

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


huggingface api: 0requests [00:00, ?requests/s]

openai api: 0requests [00:00, ?requests/s]

In [4]:
Tru().start_dashboard(_dev=Path.cwd().parent.parent)
# Tru().stop_dashboard()


  You can now view your Streamlit app in your browser.

  Network URL: http://192.168.1.70:8501
  External URL: http://73.222.135.127:8501

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


openai api: 0requests [00:00, ?requests/s]
huggingface api: 0requests [00:00, ?requests/s]

cohere api: 0requests [00:00, ?requests/s]2023-06-01 17:32:56.425 Uncaught app exception
Traceback (most recent call last):
  File "/home/piotrm/anaconda3/envs/py38_trulens_eval/lib/python3.8/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 565, in _run_script
    exec(code, module.__dict__)
  File "/home/piotrm/repos/trulens/trulens_eval/trulens_eval/pages/Progress.py", line 43, in <module>
    feedbacks = lms.get_feedback(status=[-1, 0, 1])
  File "/home/piotrm/repos/trulens/trulens_eval/trulens_eval/tru_db.py", line 545, in get_feedback
    c.execute(query, vars)
sqlite3.OperationalError: no such column: f.error


In [ ]:
Tru().db._clear_tables()

In [ ]:
conn, c = Tru().db._connect()

In [ ]:
c.execute("select * from records")

In [ ]:
for row in c.fetchall():
    print(row)

In [ ]:
row[4]

In [ ]:
thread = Tru().start_evaluator()
# Tru().stop_evaluator()
# Tru().reset_database()

In [ ]:
tc = get_or_make_chain(cid=None, selector=0)

In [ ]:
tc.dict()

In [3]:
selectors = [0,1,3,4]
messages = ["Who is Shayak?", "Wer ist Shayak?", "Kim jest Shayak?", "¿Quién es Shayak?", "Was ist QII?", "Co jest QII?"]

selectors = selectors[0:1]
messages = messages[0:1]

def test_bot(selector, question):
    print(selector, question)
    chain = get_or_make_chain(cid=question + str(selector), selector=selector)
    answer = get_answer(chain=chain, question=question)
    return answer

results = []

for s in selectors:
    for m in messages:
        results.append(TP().promise(test_bot, selector=s, question=m))

TP().finish()

Finishing 1 task(s) 0 Who is Shayak?
'Starting a new conversation with 0/default.'
✅ chain 0/default -> default.sqlite
✅ feedback def. feedback_definition_hash_025a043cd82d505769e565d5c68254af -> default.sqlite
✅ feedback def. feedback_definition_hash_8b0a16444ef8795e169e19c618918b71 -> default.sqlite
✅ feedback def. feedback_definition_hash_a9267df735153b749853787f81f434e6 -> default.sqlite
inputs= {'question': 'Who is Shayak?', 'chat_history': 'Human: Who is Shayak?\nAI:  Shayak is a computer scientist who obtained his PhD in Computer Science from Carnegie Mellon University and BTech in Computer Science from the Indian Institute of Technology, Delhi. He has been building systems and leading research to make machine learning and big data systems more explainable, privacy compliant, and fair for the past 10 years. He is also a member of the Bank of England’s AI Public-Private Forum and the OECD Global Partnership on AI.'}
creating record with
{'main_input': 'Who is Shayak?', 'main_outp

0

# Local Model Testing

In [ ]:
from langchain import ConversationChain, HuggingFacePipeline
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

base_model = LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native", device_map='auto')

# Convert the model's parameters to a higher precision data type
base_model = base_model.float()

pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# We are going to set the memory to go back 4 turns
# window_memory = ConversationBufferWindowMemory(k=4)

In [ ]:
conversation = ConversationChain(
    llm=local_llm, 
    verbose=True
)

In [ ]:
# Setup some feedback functions.

from trulens_eval import tru_feedback
from trulens_eval.tru_db import Record
import numpy as np

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Toxicity (of the response):
f_toxic = tru_feedback.Feedback(hugs.not_toxic).on_response()

# Language match (between prompt and response):
f_lang_match = tru_feedback.Feedback(hugs.language_match).on(
    text1="prompt", text2="response"
)

# Question to answer relevance:
f_relevance = tru_feedback.Feedback(openai.relevance).on(
    prompt="input", response="output"
)

# Question to context piece relevance:
#f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
#    question="input",
#    statement=Record.chain.combine_docs_chain._call.args.inputs.input_documents
#).on_multiple(
#    multiarg="statement", each_query=Record.page_content, agg=np.min
#)

feedbacks=[
    # f_toxic, 
    f_lang_match, 
    # f_relevance, 
    # f_qs_relevance
]

In [ ]:
tc = Tru().Chain(conversation, chain_id="llama-7b-hf-convo", feedbacks=feedbacks)

In [ ]:
res, record = tc.call_with_record("What is the meaning of life?")

In [ ]:
res, record = tc.call_with_record("Who is Nora?")